In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data prepration:

In [ ]:
import os
import pandas as pd

def generate_audio_text_files(directory_path, output_directory, csv_path):
    # Load the transcription CSV
    try:
        transcription_df = pd.read_csv(csv_path)
        transcription_dict = dict(zip(transcription_df['Id'], transcription_df['Transcription']))
        print(f"Loaded {len(transcription_dict)} transcriptions from CSV.")
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        return

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)
    print(f"Output directory created at: {output_directory}")

    # Open the files to write paths and text content
    with open(os.path.join(output_directory, 'audio_paths'), 'w') as audio_file, \
         open(os.path.join(output_directory, 'text'), 'w') as text_file:

        file_count = 0  # Track the number of processed files
        examples = []  # Store a few examples to print later

        # Loop through all files in the directory
        for filename in sorted(os.listdir(directory_path)):
            if filename.endswith('.wav'):
                # Extract numerical ID from filename
                utt_id = os.path.splitext(filename)[0]  # removes '.wav' extension
                file_path = os.path.join(directory_path, filename)

                # Get transcription for this utt_id from the dictionary
                transcription = transcription_dict.get(int(utt_id))

                if transcription is None:
                    # Delete the file if transcription is not found
                    print(f"No transcription found for file {filename} (ID: {utt_id}). Deleting file.")
                    os.remove(file_path)
                    continue

                # Write to audio_paths file with full path
                audio_file.write(f"{utt_id} {file_path}\n")

                # Write to text file
                text_file.write(f"{utt_id} {transcription}\n")

                # Store example
                if file_count < 5:  # Only store the first few examples
                    examples.append((utt_id, file_path, transcription))

                # Print status
                print(f"Processed file {filename}: {utt_id} -> {file_path}")
                file_count += 1

        if file_count == 0:
            print("No .wav files were processed.")
        else:
            print(f"Total .wav files processed: {file_count}")

            # Print a few examples
            print("\nExamples of processed files:")
            for example in examples:
                print(f"ID: {example[0]}, Path: {example[1]}, Transcription: {example[2]}")

# Usage
input_directory = '/content/drive/MyDrive/lifecycle/281474976710731'
output_directory = '/content/drive/MyDrive/sample_data_lifecycle'
csv_path = '/content/drive/MyDrive/Braj_lifecycle(3).csv'
generate_audio_text_files(input_directory, output_directory, csv_path)


Loaded 400 transcriptions from CSV.
Output directory created at: /content/drive/MyDrive/sample_data_lifecycle
Processed file 281474976733315.wav: 281474976733315 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733315.wav
Processed file 281474976733316.wav: 281474976733316 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733316.wav
Processed file 281474976733317.wav: 281474976733317 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733317.wav
Processed file 281474976733318.wav: 281474976733318 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733318.wav
Processed file 281474976733319.wav: 281474976733319 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733319.wav
Processed file 281474976733320.wav: 281474976733320 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733320.wav
Processed file 281474976733321.wav: 281474976733321 -> /content/drive/MyDrive/lifecycle/281474976710731/281474976733321.wav
Processed file 2814749

In [ ]:
!python /content/drive/MyDrive/data_prep.py --source_data_dir /content/drive/MyDrive/sample_data_lifecycle --output_data_dir /content/drive/MyDrive/sample_data_lifecycle_1

Casting the dataset: 100% 400/400 [00:00<00:00, 106003.77 examples/s]
Saving the dataset (1/1 shards): 100% 400/400 [00:05<00:00, 73.59 examples/s]
Data preparation done


fine tune:

In [ ]:
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.9 MB/s eta 0:00:00


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.2 MB/s eta 0:00:00


In [ ]:
ngpu=1  # Set to 1 for single-GPU training in Colab

!torchrun --nproc_per_node=$ngpu /content/drive/MyDrive/just_dw1.py \
--model_name vasista22/whisper-hindi-small \
--language Hindi \
--sampling_rate 16000 \
--num_proc 2 \
--train_strategy epoch \
--learning_rate 3e-3 \
--warmup 1000 \
--train_batchsize 8 \
--eval_batchsize 8 \
--num_epochs 20 \
--resume_from_ckpt None \
--output_dir /content/drive/MyDrive/dir_op \
--train_datasets /content/drive/MyDrive/sample_data_1/ \
--eval_datasets /content/drive/MyDrive/sample_data_lifecycle_1


2024-11-12 12:31:15.486683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 12:31:15.509247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 12:31:15.515212: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 12:31:15.529682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 12:31:17.028533: W tensorflow/comp

Single audio transcrption:

In [ ]:
!python3 /content/drive/MyDrive/transcribe_audio2.py \
--is_public_repo False \
--ckpt_dir "/content/drive/MyDrive/openai-whisper-small-braj/checkpoint-2640" \
--temp_ckpt_folder "temp" \
--path_to_audio /content/drive/MyDrive/translation/281474976710730/281474976714320.wav \
--language "hi" \
--device 0


2024-11-13 16:49:34.052483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 16:49:34.082534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 16:49:34.092073: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 16:49:34.113898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 16:49:35.543659: W tensorflow/comp